In [1]:
import torch
import torch.nn as nn

GPT_CONFIG_124M = {
    "vocab_size": 50257,    
    "context_length": 1024, 
    "emb_dim": 768,         
    "n_heads": 12,          
    "n_layers": 12,         
    "drop_rate": 0.1,       
    "qkv_bias": True       
}


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\Niteesh SK\OneDrive\Desktop\medical_llm\venv\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "c:\Users\Niteesh SK\OneDrive\Desktop\medical_llm\venv\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Users\Niteesh SK\OneDrive\Desktop\medical_llm\venv\Lib\site-packages\ipykernel\kernelapp.py", lin

In [2]:
class multiheadv2(nn.Module):
    def __init__(self,d_in,d_out,context_length,dropout,attention_head,boolbias):
        super().__init__()
        self.head_dim=d_out//attention_head
        self.d_out=d_out
        self.attention_head=attention_head

        self.W_query = nn.Linear(d_in, d_out, bias=boolbias)
        self.W_key = nn.Linear(d_in, d_out, bias=boolbias)
        self.W_value = nn.Linear(d_in, d_out, bias=boolbias)

        self.out_proj = nn.Linear(d_out, d_out)  
        self.dropout = nn.Dropout(dropout)
        self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length), diagonal=1))

    def forward(self,x):
        b,num_token,d_out=x.shape

        keys = self.W_key(x) 
        queries = self.W_query(x)
        values = self.W_value(x)

        keys=keys.view(b,num_token,self.attention_head,self.head_dim)
        queries=queries.view(b,num_token,self.attention_head,self.head_dim)
        values=values.view(b,num_token,self.attention_head,self.head_dim)

        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        attn_score=queries @ keys.transpose(2,3)

        mask_bool = self.mask.bool()[:num_token, :num_token]
        attn_score.masked_fill_(mask_bool, -torch.inf)
        
        attn_weights = torch.softmax(attn_score / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)


        context_vec = (attn_weights @ values).transpose(1, 2) 
        
        context_vec = context_vec.contiguous().view(b, num_token, self.d_out)
        context_vec = self.out_proj(context_vec) # optional projection

        return context_vec
            



In [3]:
class LayerNorm(nn.Module):
    def __init__(self,emb_dim):
        super().__init__()
        self.eps=1e-5
        self.scale_params=nn.Parameter(torch.ones(emb_dim))
        self.shift_params=nn.Parameter(torch.zeros(emb_dim))
    
    def forward(self,x):
        mean=x.mean(dim=-1,keepdim=True)
        var=x.var(dim=-1,keepdim=True,unbiased=False)
        norm=(x-mean)/torch.sqrt(var+self.eps)
        return norm*self.scale_params+ self.shift_params
    
        

In [4]:
class GELU(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(torch.sqrt(torch.tensor(2.0 / torch.pi)) *(x + 0.044715 * torch.pow(x, 3))
        ))
    

class feedforward(nn.Module):
    def __init__(self,config):
        super().__init__()
        self.layers=nn.Sequential(
            nn.Linear(config['emb_dim'],config['emb_dim']*4),
            GELU(),
            nn.Linear(config['emb_dim']*4,config['emb_dim']),

        )
    
    def forward(self,x):
        return self.layers(x)

In [5]:
class TransformerBlock(nn.Module):
    def __init__ (self,config):
        super().__init__()
        self.attn=multiheadv2(d_in=config['emb_dim'],d_out=config['emb_dim'],context_length=config['context_length'],dropout=config['drop_rate'],attention_head=config['n_heads'],boolbias=config['qkv_bias'])
        self.Layernorm1=LayerNorm(emb_dim=config['emb_dim'])
        self.Layernorm2=LayerNorm(emb_dim=config['emb_dim'])
        self.feedforw=feedforward(config=config)
        self.dropout=nn.Dropout(config['drop_rate'])
    
    def forward(self,x):
        ## attnetion block
        skip=x
        x=self.Layernorm1(x)
        x=self.attn(x)
        x=self.dropout(x)
        x=x+skip

        ## feed forward nn block
        skip=x
        x=self.Layernorm2(x)
        x=self.feedforw(x)
        x=self.dropout(x)
        x=x+skip

        return x


In [6]:
class GPT_2(nn.Module):
    def __init__ (self,cfg):
        super().__init__()
        self.token_emb=nn.Embedding(cfg['vocab_size'],cfg["emb_dim"])
        self.pos_emb=nn.Embedding(cfg['context_length'],cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])
        self.trf_blocks = nn.Sequential(
            *[TransformerBlock(cfg) for _ in range(cfg["n_layers"])])
        
        self.final_norm = LayerNorm(cfg["emb_dim"])
        self.out_head = nn.Linear(
            cfg["emb_dim"], cfg["vocab_size"], bias=False
        )
    
    def forward(self,inputidx):
        batch_size,seq=inputidx.shape
        tokens=self.token_emb(inputidx)
        pos_embeds = self.pos_emb(torch.arange(seq, device=inputidx.device))
        x=tokens+pos_embeds
        x=self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits
    



In [7]:
def generate(model, idx, max_token_size, context_Size, top_k=5, temperature=1.0):
    for _ in range(max_token_size):
        idx_split = idx[:, -context_Size:]

        with torch.no_grad():
            logits = model(idx_split)

        logits_last = logits[:, -1, :]
        
        logits_last = logits_last / temperature
        
       
        values, indices = torch.topk(logits_last, k=top_k, dim=-1)
        probs = torch.softmax(values, dim=-1)
        sampled_idx = torch.multinomial(probs, num_samples=1)
        maxi = torch.gather(indices, -1, sampled_idx)
       
    
        idx = torch.cat((idx, maxi), dim=1)
    
    return idx

import tiktoken
tokenizer = tiktoken.get_encoding("gpt2")
def text_to_token_ids(text, tokenizer):
    encoded = tokenizer.encode(text, allowed_special={'<|endoftext|>'})
    encoded_tensor = torch.tensor(encoded).unsqueeze(0) 
    return encoded_tensor

def token_ids_to_text(token_ids, tokenizer):
    flat = token_ids.squeeze(0) 
    return tokenizer.decode(flat.tolist())


In [8]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [9]:
## This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [10]:
import os
import json
import numpy as np
import tensorflow as tf

GPT2_MODEL_PATH = r"C:\Users\Niteesh SK\OneDrive\Desktop\medical_llm\124M\124M"

def load_gpt2_params_from_tf_ckpt(ckpt_path, settings):
    params = {"blocks": [{} for _ in range(settings["n_layer"])]}
    for name, _ in tf.train.list_variables(ckpt_path):
        variable_array = np.squeeze(tf.train.load_variable(ckpt_path, name))
        variable_name_parts = name.split("/")[1:]  # Skip 'model/' prefix
        target_dict = params
        if variable_name_parts[0].startswith("h"):
            layer_number = int(variable_name_parts[0][1:])
            target_dict = params["blocks"][layer_number]
        for key in variable_name_parts[1:-1]:
            target_dict = target_dict.setdefault(key, {})
        last_key = variable_name_parts[-1]
        target_dict[last_key] = variable_array
    return params

def load_gpt2(model_dir):
    tf_ckpt_path = tf.train.latest_checkpoint(model_dir)
    if tf_ckpt_path is None:
        raise FileNotFoundError(f"No checkpoint found in {model_dir}")
    hparams_path = os.path.join(model_dir, "hparams.json")
    if not os.path.exists(hparams_path):
        raise FileNotFoundError(f"hparams.json not found in {model_dir}")
    settings = json.load(open(hparams_path))
    params = load_gpt2_params_from_tf_ckpt(tf_ckpt_path, settings)
    return settings, params

# Load GPT-2 weights
settings, params = load_gpt2(GPT2_MODEL_PATH)
print("Loaded GPT-2 124M checkpoint successfully!")


Loaded GPT-2 124M checkpoint successfully!


In [11]:
import torch
import torch.nn as nn

GPT_CONFIG_124M = {
    "vocab_size": 50257,    
    "context_length": 1024, 
    "emb_dim": 768,         
    "n_heads": 12,          
    "n_layers": 12,         
    "drop_rate": 0.1,       
    "qkv_bias": True       
}

In [12]:
def assign(left, right):
    if left.shape != right.shape:
        raise ValueError(f"Shape mismatch. Left: {left.shape}, Right: {right.shape}")
    return torch.nn.Parameter(torch.tensor(right))

In [13]:
import numpy as np
import torch

def load_weights_into_gpt(gpt, params):
    gpt.pos_emb.weight = assign(gpt.pos_emb.weight, params['wpe'])
    gpt.token_emb.weight = assign(gpt.token_emb.weight, params['wte'])
    
    for b in range(len(params["blocks"])):
        q_w, k_w, v_w = np.split(params["blocks"][b]["attn"]["c_attn"]["w"], 3, axis=-1)
        gpt.trf_blocks[b].attn.W_query.weight = assign(gpt.trf_blocks[b].attn.W_query.weight, q_w.T)
        gpt.trf_blocks[b].attn.W_key.weight = assign(gpt.trf_blocks[b].attn.W_key.weight, k_w.T)
        gpt.trf_blocks[b].attn.W_value.weight = assign(gpt.trf_blocks[b].attn.W_value.weight, v_w.T)

        q_b, k_b, v_b = np.split(params["blocks"][b]["attn"]["c_attn"]["b"], 3, axis=-1)
        gpt.trf_blocks[b].attn.W_query.bias = assign(gpt.trf_blocks[b].attn.W_query.bias, q_b)
        gpt.trf_blocks[b].attn.W_key.bias = assign(gpt.trf_blocks[b].attn.W_key.bias, k_b)
        gpt.trf_blocks[b].attn.W_value.bias = assign(gpt.trf_blocks[b].attn.W_value.bias, v_b)

        gpt.trf_blocks[b].attn.out_proj.weight = assign(gpt.trf_blocks[b].attn.out_proj.weight, params["blocks"][b]["attn"]["c_proj"]["w"].T)
        gpt.trf_blocks[b].attn.out_proj.bias = assign(gpt.trf_blocks[b].attn.out_proj.bias, params["blocks"][b]["attn"]["c_proj"]["b"])

        gpt.trf_blocks[b].feedforw.layers[0].weight = assign(gpt.trf_blocks[b].feedforw.layers[0].weight, params["blocks"][b]["mlp"]["c_fc"]["w"].T)
        gpt.trf_blocks[b].feedforw.layers[0].bias = assign(gpt.trf_blocks[b].feedforw.layers[0].bias, params["blocks"][b]["mlp"]["c_fc"]["b"])
        gpt.trf_blocks[b].feedforw.layers[2].weight = assign(gpt.trf_blocks[b].feedforw.layers[2].weight, params["blocks"][b]["mlp"]["c_proj"]["w"].T)
        gpt.trf_blocks[b].feedforw.layers[2].bias = assign(gpt.trf_blocks[b].feedforw.layers[2].bias, params["blocks"][b]["mlp"]["c_proj"]["b"])

        gpt.trf_blocks[b].Layernorm1.scale_params = assign(gpt.trf_blocks[b].Layernorm1.scale_params, params["blocks"][b]["ln_1"]["g"])
        gpt.trf_blocks[b].Layernorm1.shift_params = assign(gpt.trf_blocks[b].Layernorm1.shift_params, params["blocks"][b]["ln_1"]["b"])
        gpt.trf_blocks[b].Layernorm2.scale_params = assign(gpt.trf_blocks[b].Layernorm2.scale_params, params["blocks"][b]["ln_2"]["g"])
        gpt.trf_blocks[b].Layernorm2.shift_params = assign(gpt.trf_blocks[b].Layernorm2.shift_params, params["blocks"][b]["ln_2"]["b"])

    gpt.final_norm.scale_params = assign(gpt.final_norm.scale_params, params["g"])
    gpt.final_norm.shift_params = assign(gpt.final_norm.shift_params, params["b"])
    gpt.out_head.weight = assign(gpt.out_head.weight, params["wte"])



In [14]:
def generate(model, idx, max_token_size, context_Size, top_k=5, temperature=1.0):
    for _ in range(max_token_size):
        idx_split = idx[:, -context_Size:]

        with torch.no_grad():
            logits = model(idx_split)

        logits_last = logits[:, -1, :]
        
        logits_last = logits_last / temperature
        
       
        values, indices = torch.topk(logits_last, k=top_k, dim=-1)
        probs = torch.softmax(values, dim=-1)
        sampled_idx = torch.multinomial(probs, num_samples=1)
        maxi = torch.gather(indices, -1, sampled_idx)
       
    
        idx = torch.cat((idx, maxi), dim=1)
    
    return idx

import tiktoken
tokenizer = tiktoken.get_encoding("gpt2")
def text_to_token_ids(text, tokenizer):
    encoded = tokenizer.encode(text, allowed_special={'<|endoftext|>'})
    encoded_tensor = torch.tensor(encoded).unsqueeze(0) 
    return encoded_tensor

def token_ids_to_text(token_ids, tokenizer):
    flat = token_ids.squeeze(0) 
    return tokenizer.decode(flat.tolist())


In [15]:
def assign(left, right, transpose=False):
    """
    Assign a NumPy array to a PyTorch parameter.
    """
    import torch

    tensor = torch.tensor(right, dtype=torch.float32)  # force float32

    # Remove extra dimensions if necessary
    while tensor.ndim > len(left.shape):
        tensor = tensor.squeeze(0)

    # Transpose if needed
    if transpose:
        tensor = tensor.T

    if left.shape != tensor.shape:
        raise ValueError(f"Shape mismatch. Left: {left.shape}, Right: {tensor.shape}")

    return torch.nn.Parameter(tensor)


In [16]:
model = GPT_2(GPT_CONFIG_124M)
load_weights_into_gpt(model, params)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


if torch.cuda.is_available():
   device = torch.device("cuda")
elif torch.backends.mps.is_available():
   device = torch.device("mps")
else:
   device = torch.device("cpu")

print(f"Using {device} device.")


model.to(device) 


Using cpu device.


GPT_2(
  (token_emb): Embedding(50257, 768)
  (pos_emb): Embedding(1024, 768)
  (drop_emb): Dropout(p=0.1, inplace=False)
  (trf_blocks): Sequential(
    (0): TransformerBlock(
      (attn): multiheadv2(
        (W_query): Linear(in_features=768, out_features=768, bias=True)
        (W_key): Linear(in_features=768, out_features=768, bias=True)
        (W_value): Linear(in_features=768, out_features=768, bias=True)
        (out_proj): Linear(in_features=768, out_features=768, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (Layernorm1): LayerNorm()
      (Layernorm2): LayerNorm()
      (feedforw): feedforward(
        (layers): Sequential(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU()
          (2): Linear(in_features=3072, out_features=768, bias=True)
        )
      )
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (1): TransformerBlock(
      (attn): multiheadv2(
        (W_query): Linear(in_features=768,

In [17]:
def compute_checksum(model):
    return sum(p.sum().item() for p in model.parameters())

checksum = compute_checksum(model)
print(f"Model weight checksum: {checksum}")


Model weight checksum: -47459.90885190666


In [18]:
import pandas as pd

df_train = pd.read_csv(r"C:\Users\Niteesh SK\OneDrive\Desktop\medical_llm\PubMed_20k_RCT\train.csv")
df_train = df_train[['abstract_text', 'line_number', 'total_lines', 'target']]
df_train['target'] = df_train['target'].astype('category').cat.codes
df_train

df_test = pd.read_csv(r"C:\Users\Niteesh SK\OneDrive\Desktop\medical_llm\PubMed_20k_RCT\test.csv")
df_test = df_test[['abstract_text', 'line_number', 'total_lines', 'target']]
df_test['target'] = df_test['target'].astype('category').cat.codes
df_test



,abstract_text,line_number,total_lines,target
0,This study analyzed liver function abnormaliti...,0,9,0
1,A post hoc analysis was conducted with the use...,1,9,4
2,Liver function tests ( LFTs ) were measured at...,2,9,4
3,Survival analyses were used to assess the asso...,3,9,4
4,The percentage of patients with abnormal LFTs ...,4,9,4
...,...,...,...,...
30130,There was a statistically significant between-...,13,18,4
30131,There were no statistically significant betwee...,14,18,4
30132,There was no significant association between s...,15,18,4
30133,No adverse effects were reported .,16,18,4


In [19]:
df_train.to_csv("train.csv", index=None)
df_test.to_csv("test.csv", index=None)

In [20]:
import torch
import pandas as pd
import tiktoken
from torch.utils.data import Dataset

class MedicalDataset(Dataset):
    def __init__(self, csv_file, tokenizer, max_length=128, pad_token_id=50256):
        self.data = pd.read_csv(csv_file).dropna()  
        self.tokenizer = tokenizer
        self.max_length = max_length if max_length is not None else 128  

        self.encoded_texts = [
            tokenizer.encode(text)[:self.max_length] for text in self.data["abstract_text"]
        ]

        self.encoded_texts = [
            encoded + [pad_token_id] * (self.max_length - len(encoded)) for encoded in self.encoded_texts
        ]

        self.labels = torch.tensor(self.data["target"].astype('category').cat.codes, dtype=torch.long)

    def __getitem__(self, idx):
        return torch.tensor(self.encoded_texts[idx], dtype=torch.long), self.labels[idx]

    def __len__(self):
        return len(self.data)

tokenizer = tiktoken.get_encoding("gpt2")


In [21]:
train_dataset = MedicalDataset(
    csv_file=r"C:\Users\Niteesh SK\OneDrive\Desktop\medical_llm\PubMed_20k_RCT\train.csv",
    max_length=None,
    tokenizer=tokenizer
)

val_dataset = MedicalDataset(
    csv_file=r"C:\Users\Niteesh SK\OneDrive\Desktop\medical_llm\PubMed_20k_RCT\train.csv",  # or a validation split
    max_length=train_dataset.max_length,
    tokenizer=tokenizer
)

test_dataset = MedicalDataset(
    csv_file=r"C:\Users\Niteesh SK\OneDrive\Desktop\medical_llm\PubMed_20k_RCT\test.csv",
    max_length=train_dataset.max_length,
    tokenizer=tokenizer
)


In [22]:
from torch.utils.data import DataLoader

num_workers = 0
batch_size = 128

torch.manual_seed(123)

train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers,
    drop_last=True,
)

val_loader = DataLoader(
    dataset=val_dataset,
    batch_size=batch_size,
    num_workers=num_workers,
    drop_last=False,
)

test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    num_workers=num_workers,
    drop_last=False,
)

In [23]:
for param in model.parameters():
    param.requires_grad = False



torch.manual_seed(123)

num_classes = 5
model.out_head = torch.nn.Linear(in_features=GPT_CONFIG_124M["emb_dim"], out_features=num_classes)

for param in model.trf_blocks[-1].parameters():
    param.requires_grad = True

for param in model.final_norm.parameters():
    param.requires_grad = True

In [24]:
def calc_accuracy_loader(data_loader, model, device, num_batches=None):
    model.eval()
    correct_predictions, num_examples = 0, 0

    if num_batches is None:
        num_batches = len(data_loader)
    else:
        num_batches = min(num_batches, len(data_loader))
    for i, (input_batch, target_batch) in enumerate(data_loader):
        if i < num_batches:
            input_batch, target_batch = input_batch.to(device), target_batch.to(device)

            with torch.no_grad():
                logits = model(input_batch)[:, -1, :]  
            predicted_labels = torch.argmax(logits, dim=-1)

            num_examples += predicted_labels.shape[0]
            correct_predictions += (predicted_labels == target_batch).sum().item()
        else:
            break
    return correct_predictions / num_examples

In [25]:
def calc_loss_batch(input_batch, target_batch, model, device):
    input_batch, target_batch = input_batch.to(device), target_batch.to(device)
    logits = model(input_batch)[:, -1, :]  
    loss = torch.nn.functional.cross_entropy(logits, target_batch)
    return loss


def calc_loss_loader(data_loader, model, device, num_batches=None):
    total_loss = 0.
    if len(data_loader) == 0:
        return float("nan")
    elif num_batches is None:
        num_batches = len(data_loader)
    else:
        num_batches = min(num_batches, len(data_loader))
    for i, (input_batch, target_batch) in enumerate(data_loader):
        if i < num_batches:
            loss = calc_loss_batch(input_batch, target_batch, model, device)
            total_loss += loss.item()
        else:
            break
    return total_loss / num_batches

In [26]:
def evaluate_model(model, train_loader, val_loader, device, eval_iter):
    model.eval()
    with torch.no_grad():
        train_loss = calc_loss_loader(train_loader, model, device, num_batches=eval_iter)
        val_loss = calc_loss_loader(val_loader, model, device, num_batches=eval_iter)
    model.train()
    return train_loss, val_loss

In [27]:
def train_classifier_simple(model, train_loader, val_loader, optimizer, device,
                            num_epochs, eval_freq, eval_iter, start_epoch=0):
    train_losses, val_losses, train_accs, val_accs = [], [], [], []
    examples_seen, global_step = 0, -1

    for epoch in range(start_epoch, num_epochs):  # <-- use start_epoch here
        model.train()

        for input_batch, target_batch in train_loader:
            optimizer.zero_grad()
            loss = calc_loss_batch(input_batch, target_batch, model, device)
            loss.backward()
            optimizer.step()
            examples_seen += input_batch.shape[0]
            global_step += 1

            if global_step % eval_freq == 0:
                train_loss, val_loss = evaluate_model(
                    model, train_loader, val_loader, device, eval_iter)
                train_losses.append(train_loss)
                val_losses.append(val_loss)
                print(f"Ep {epoch+1} (Step {global_step:06d}): "
                      f"Train loss {train_loss:.3f}, Val loss {val_loss:.3f}")

        train_accuracy = calc_accuracy_loader(train_loader, model, device, num_batches=eval_iter)
        val_accuracy = calc_accuracy_loader(val_loader, model, device, num_batches=eval_iter)
        print(f"Training accuracy: {train_accuracy*100:.2f}% | "
              f"Validation accuracy: {val_accuracy*100:.2f}%")
        train_accs.append(train_accuracy)
        val_accs.append(val_accuracy)

    return train_losses, val_losses, train_accs, val_accs, examples_seen


In [39]:
import time

start_time = time.time()

torch.manual_seed(123)

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5, weight_decay=0.1)
model.to(device) 

num_epochs = 10
train_losses, val_losses, train_accs, val_accs, examples_seen = train_classifier_simple(
    model, train_loader, val_loader, optimizer, device,
    num_epochs=num_epochs, eval_freq=50, eval_iter=5,
)

end_time = time.time()
execution_time_minutes = (end_time - start_time) / 60
print(f"Training completed in {execution_time_minutes:.2f} minutes.")



KeyboardInterrupt: 

In [35]:
# inside your training loop after each epoch
checkpoint = {
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict()
}
torch.save(checkpoint, "checkpoint1.pth")


In [ ]:
total_epochs = 10  # total epochs you want to run
checkpoint = torch.load("checkpoint1.pth")
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
model.to(device)

remaining_epochs = total_epochs

train_losses, val_losses, train_accs, val_accs, examples_seen = train_classifier_simple(
    model, train_loader, val_loader, optimizer, device,
    num_epochs=remaining_epochs,
    eval_freq=50,
    eval_iter=5,
)

Ep 1 (Step 000000): Train loss 0.964, Val loss 1.019
Ep 1 (Step 000050): Train loss 0.882, Val loss 0.962
Ep 1 (Step 000100): Train loss 0.836, Val loss 0.905
Ep 1 (Step 000150): Train loss 0.842, Val loss 0.864
Ep 1 (Step 000200): Train loss 0.783, Val loss 0.852
Ep 1 (Step 000250): Train loss 0.734, Val loss 0.831
Ep 1 (Step 000300): Train loss 0.725, Val loss 0.808
Ep 1 (Step 000350): Train loss 0.708, Val loss 0.810
Ep 1 (Step 000400): Train loss 0.779, Val loss 0.793
Ep 1 (Step 000450): Train loss 0.738, Val loss 0.792


In [ ]:
train_accuracy = calc_accuracy_loader(train_loader, model, device)
val_accuracy = calc_accuracy_loader(val_loader, model, device)
test_accuracy = calc_accuracy_loader(test_loader, model, device)

print(f"Training accuracy: {train_accuracy*100:.2f}%")
print(f"Validation accuracy: {val_accuracy*100:.2f}%")
print(f"Test accuracy: {test_accuracy*100:.2f}%")

KeyboardInterrupt: 

In [ ]:
df_temp=pd.read_csv('/kaggle/input/pubmed-200k-rtc/PubMed_20k_RCT/train.csv')
label_mapping = dict(enumerate(df_temp["target"].astype("category").cat.categories))

def classify_review(text, model, tokenizer, device, label_mapping, max_length=128, pad_token_id=50256):
    model.eval()
    
    input_ids = tokenizer.encode(text)[:max_length]
    input_ids += [pad_token_id] * (max_length - len(input_ids))
    input_tensor = torch.tensor(input_ids, device=device).unsqueeze(0)

    with torch.no_grad():
        logits = model(input_tensor)[:, -1, :]  
    predicted_label = torch.argmax(logits, dim=-1).item()

    return label_mapping[predicted_label]  




In [ ]:
text_1 = "This study evaluates the effectiveness of a new treatment for knee arthritis."
print(classify_review(text_1, model, tokenizer, device, label_mapping, max_length=train_dataset.max_length))

text_2="Patients who received the new drug showed a 40% reduction in pain compared to the control group."
print(classify_review(text_2, model, tokenizer, device, label_mapping, max_length=train_dataset.max_length))

text_3="Participants were randomly assigned to either the treatment or placebo group and monitored for 12 weeks"
print(classify_review(text_3, model, tokenizer, device, label_mapping, max_length=train_dataset.max_length))

text_4="Osteoarthritis is a leading cause of disability among older adults, affecting millions worldwide"
print(classify_review(text_4, model, tokenizer, device, label_mapping, max_length=train_dataset.max_length))

text_5="Although the results were promising, the long-term effects of the treatment remain unknown and should be investigated further."
print(classify_review(text_5, model, tokenizer, device, label_mapping, max_length=train_dataset.max_length))
